In [35]:
import PyPDF2
import re
import pandas as pd
import json

def parse_timetable(file_path, start_page):
    # read your file
    pdf = PyPDF2.PdfReader(open(file_path, 'rb'))

    # strings to clenup
    replace = ["Syllabus/Component Code Duration Date Session",
               "Sessions:  AM morning        PM afternoon        EV evening\nIG Cambridge IGCSE    OL Cambridge O Level    \nAS Cambridge International AS Level    AL Cambridge International A Level    \nCambridge Final Exam Timetable June 2023", "Syllabus view (A–Z)", "Sessions:  AM morning        PM afternoon        \nIG Cambridge IGCSE    \nAS Cambridge International AS Level    AL Cambridge International A Level    \nCambridge Final Exam Timetable June 2023", "Sessions:  AM morning        PM afternoon        \nIG Cambridge IGCSE       \nAS Cambridge International AS Level    AL Cambridge International A Level    \nCambridge Final Exam Timetable June 2023", "Sessions:  AM morning        PM afternoon        \nIG Cambridge IGCSE    9–1 Cambridge IGCSE (9–1)    OL Cambridge O Level    \nAS Cambridge International AS Level    AL Cambridge International A Level    \nCambridge Final Exam Timetable June 2023", "Sessions:  AM morning        PM afternoon        EV evening\nIG Cambridge IGCSE    OL Cambridge O Level    \nAS Cambridge International AS Level    AL Cambridge International A Level    \nCambridge Final Exam Timetable June 2023", "Sessions:  AM morning        PM afternoon        EV evening\nIG Cambridge IGCSE    OL Cambridge O Level    \nAS Cambridge International AS Level    AL Cambridge International A Level    \nCambridge Final Exam Timetable June 2023", "Sessions:  AM morning        PM afternoon        \nIG Cambridge IGCSE    \nAS Cambridge International AS Level    AL Cambridge International A Level    \nCambridge Final Exam Timetable June 2023"]

    data = []

    for i in range(start_page, len(pdf.pages)):
        text = pdf.pages[i].extract_text()
        for replace_line in replace:
            text = text.replace(replace_line, '')
        temp = list(filter(None, text[:-3].splitlines()))

        text = []

        def dontJoin(x): return x.split()[0] == 'Cambridge' or len(
            x) == 1 or x[-2:] in ['AM', 'PM', 'EV']

        flag = False
        for i in range(len(temp)):
            if dontJoin(temp[i]):
                if flag or len(temp[i]) == 1 or temp[i][-9:] == 'Continued':
                    flag = False
                    continue
                text.append(temp[i])
            else:
                text.append(temp[i] + " " + temp[i+1])
                flag = True
                i += 1

        data.extend(text)

    data_string = "\n".join(data)

    titles = re.findall(r'Cambridge[a-zA-Z ]*\n', data_string)

    # split the data_string based on the titles
    data = re.split(r'Cambridge[a-zA-Z ]*\n', data_string)[1:]
    
    data = [[[titles[i].strip(), x, *re.findall(
            r'(.*)[ ]?([0-9]{4}\/[0-9]{2}) ([0-9]*?h?[ ]?[0-9]*?m?) (.*)', x)] for x in sorted(d.splitlines())]
            for i, d in enumerate(data)]

    # flatten data
    def flatten(l): return [item for sublist in l for item in sublist]

    flat_data = flatten(data)
    
    for i in flat_data:
        if len(i) < 3:
            print(i, "hmm check here")
            flat_data.remove(i)
    flat_data = [[i[0], *i[2]] for i in flat_data]

    # convert the data_map to a dataframe
    df = pd.DataFrame(flat_data)
    df.columns = ['type', 'subject', 'code', 'duration', 'date']

    return df

In [36]:
start_page_list = [9, 9, 10, 9, 9, 9]
for idx in range(1,7):
    print(f'Parsing zone {idx}...')
    df = parse_timetable(f'pdfs/z{idx}.pdf', start_page_list[idx-1])
    
    df.to_csv(f'csv/zone{idx}.csv', index=False)

    grouped_data = df.groupby(df['code'].str[:4])

    # make a list of the grouped data
    grouped_data_list = [grouped_data.get_group(x) for x in grouped_data.groups]

    grouped_data = []

    for group in grouped_data_list:
        # get the common substring of the subjects
        # get a list of the subjects in the group
        subjects = group['subject'].tolist()

        # get the common substring of the subjects
        commonSubstring = ''
        
        # get the shortest subject
        shortestSubject = min(subjects, key=len)

        # iterate through the characters of the shortest subject
        for i in range(len(shortestSubject)):
            # get the current character
            currentChar = shortestSubject[i]
            # check if the current character is the same in all the subjects
            if all([subject[i] == currentChar for subject in subjects]) and (currentChar.isalpha() or currentChar == ' '):
                # add the current character to the common substring
                commonSubstring += currentChar
            else:
                # break the loop if the current character is not the same in all the subjects
                break

        # convert the group to a dictionary
        group = group.sort_values(by=['code']).to_dict('records')
        # add the group to the output data
        grouped_data.append({
            'code': group[0]['code'][:4],
            'commonSubstring': commonSubstring.strip(),
            'group': group
        })
        
    # save the output data to a json file
    json.dump(grouped_data, open(f'json/zone{idx}.json', 'w'), indent=2)


Parsing zone 1...
Parsing zone 2...
Parsing zone 3...
['Cambridge IGCSE', 'Cambridge IGCSE (9–1)'] hmm check here
Parsing zone 4...
Parsing zone 5...
Parsing zone 6...
